# Transfer Learning with VGG16 and Keras
https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4

In [1]:
import tensorflow as tf
from tensorflow import keras

import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

In [2]:
# Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], # Train test split
    batch_size=-1,
    as_supervised=True, # Include labels
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
# Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

In [4]:
# Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [5]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Loading VGG16 model
base_model = VGG16(weights="imagenet",
                   include_top=False, # remove the classification layer that was trained on ImageNet dataset
                   input_shape=train_ds[0].shape)
base_model.trainable = False # Not trainable weights

58889256/58889256 [==============================] - 4s 0us/step


In [6]:
# Preprocessing input (normalize the input data)
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

In [7]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [8]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

In [9]:
model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)


model.fit(train_ds, train_labels, epochs=50, validation_split=0.20, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 18s 109ms/step - loss: 1.7668 - accuracy: 0.4944 - val_loss: 1.1364 - val_accuracy: 0.6128
Epoch 2/50
65/65 [==============================] - 5s 84ms/step - loss: 0.7656 - accuracy: 0.7465 - val_loss: 1.0143 - val_accuracy: 0.6342
Epoch 3/50
65/65 [==============================] - 5s 80ms/step - loss: 0.5034 - accuracy: 0.8200 - val_loss: 1.0022 - val_accuracy: 0.6556
Epoch 4/50
65/65 [==============================] - 5s 85ms/step - loss: 0.3757 - accuracy: 0.8710 - val_loss: 1.0076 - val_accuracy: 0.6654
Epoch 5/50
65/65 [==============================] - 5s 84ms/step - loss: 0.2545 - accuracy: 0.9129 - val_loss: 0.9985 - val_accuracy: 0.6926
Epoch 6/50
65/65 [==============================] - 5s 80ms/step - loss: 0.1877 - accuracy: 0.9426 - val_loss: 1.0383 - val_accuracy: 0.6848
Epoch 7/50
65/65 [==============================] - 5s 81ms/step - loss: 0.1350 - accuracy: 0.9645 - val_loss: 1.1475 - val_accuracy: 0.6790
Epoch 8/50


## Extra: comparing to hand-made model

In [12]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [13]:
hand_made_model = Sequential()
hand_made_model.add(Rescaling(1./255, input_shape=(150,150,3)))

hand_made_model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
hand_made_model.add(layers.MaxPooling2D(3))

hand_made_model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
hand_made_model.add(layers.MaxPooling2D(2))

hand_made_model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
hand_made_model.add(layers.MaxPooling2D(2))

hand_made_model.add(layers.Flatten())
hand_made_model.add(layers.Dense(50, activation='relu'))
hand_made_model.add(layers.Dense(20, activation='relu'))
hand_made_model.add(layers.Dense(5, activation='softmax'))

In [14]:
hand_made_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

hand_made_model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 8s 41ms/step - loss: 1.4916 - accuracy: 0.3144 - val_loss: 1.3609 - val_accuracy: 0.3774
Epoch 2/50
65/65 [==============================] - 2s 28ms/step - loss: 1.3086 - accuracy: 0.4268 - val_loss: 1.2266 - val_accuracy: 0.4708
Epoch 3/50
65/65 [==============================] - 2s 30ms/step - loss: 1.2357 - accuracy: 0.4710 - val_loss: 1.3009 - val_accuracy: 0.4533
Epoch 4/50
65/65 [==============================] - 1s 22ms/step - loss: 1.1737 - accuracy: 0.4988 - val_loss: 1.1945 - val_accuracy: 0.4533
Epoch 5/50
65/65 [==============================] - 1s 19ms/step - loss: 1.0892 - accuracy: 0.5436 - val_loss: 1.1360 - val_accuracy: 0.5292
Epoch 6/50
65/65 [==============================] - 1s 19ms/step - loss: 1.0268 - accuracy: 0.5562 - val_loss: 1.1439 - val_accuracy: 0.5350
Epoch 7/50
65/65 [==============================] - 1s 19ms/step - loss: 0.9627 - accuracy: 0.5971 - val_loss: 1.1021 - val_accuracy: 0.5370
Epoch 8/50
65